<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/490_EPOv2_nodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

These two nodes are **foundational**, and they’re doing more work than they might appear to at first glance. I’ll explain them as **architecture components**, not as Python functions.

---

# Goal & Planning Nodes — Explained

## What These Nodes Do in the System

The **goal node** and **planning node** together define the agent’s **intent and execution contract**.

They answer two critical questions *before* any data is touched:

1. **What is this agent trying to accomplish right now?**
2. **What exact steps is it allowed to take to do so?**

This is a deliberate design choice that separates your agent from most LLM-based systems, which tend to jump straight into analysis without an explicit plan.

In your orchestrator, **nothing happens until the goal is defined and the plan is approved**.

---

## Goal Node: Defining Intent Explicitly

### What the goal node does

The `goal_node` translates a simple input (`experiment_id` or not) into a **structured, machine-readable objective**.

Instead of relying on implicit intent, it produces a clear goal object that includes:

* the objective
* the scope of analysis
* which experiment (if any) is in focus
* the specific business dimensions that matter

This goal object becomes the **north star** for every downstream node.

---

### Why this matters

Most agents implicitly infer goals from prompts.
Your agent **declares its goal explicitly**.

That gives you:

* deterministic behavior
* easier debugging
* safer automation
* clearer audit trails

If something goes wrong later, you can always ask:

> “What goal was the agent executing?”

And the answer is right there in state.

---

### Single-experiment vs portfolio-wide goals

This branching is subtle but important.

* **Single experiment**

  * Narrow scope
  * Deep statistical and ROI focus
  * Tactical decision-making

* **Portfolio-wide**

  * Broad scope
  * Coverage, gaps, risk, learning
  * Executive-level synthesis

By encoding this distinction early, you avoid bloating downstream logic with special cases. Every later node can simply read `goal["scope"]` and behave accordingly.

This is **clean orchestration design**.

---

## Planning Node: Turning Intent Into a Contract

### What the planning node does

The `planning_node` converts the abstract goal into a **step-by-step execution plan**.

Each step clearly defines:

* what happens
* what it depends on
* what it produces

This is not just a plan — it’s a **workflow contract**.

---

### Why this matters

This is where your agent stops being a “smart script” and becomes a **workflow orchestrator**.

Because the plan is:

* explicit
* ordered
* dependency-aware
* output-defined

You can:

* validate execution before running
* visualize the workflow
* skip or rerun steps safely
* enforce governance (e.g., block later steps if earlier ones fail)

This is the same logic used in production data pipelines — just applied to agent workflows.

---

## MVP-first, but extensible by design

Notice what you **did not** do:

* No LLM planning
* No dynamic reasoning
* No hidden logic

Instead:

* Fixed templates
* Clear step definitions
* Business-aligned outputs

This makes the system:

* predictable
* testable
* explainable

Later, you *could* introduce adaptive planning — but you don’t need it now. This is exactly the right MVP tradeoff.

---

## Why the Two Plans Are Different (and That’s a Feature)

The **single-experiment plan** is:

* short
* linear
* decision-focused

The **portfolio plan** adds:

* coverage analysis
* cross-experiment insights
* aggregation logic

This mirrors how real organizations operate:

* analysts deep-dive individual initiatives
* executives care about the portfolio

Your agent reflects that reality directly in code.

---

## Outputs as First-Class Citizens

Each step explicitly lists its outputs.

That’s not just documentation — it’s a **design guarantee**.

It ensures:

* downstream nodes know what to expect
* missing data can be detected early
* partial execution is possible
* observability is built in

This is one of the strongest signals that this system was designed for **trust and control**, not speed hacks.

---

## How This Fits the Bigger Architecture

Together, these nodes establish:

* **Intent** → what we’re doing
* **Plan** → how we’re allowed to do it
* **State** → what we know at each step

Everything else in the agent simply executes against this contract.

That’s why this design scales:

* more nodes can be added safely
* governance rules slot in naturally
* LLMs can be added *later* as enhancers, not decision-makers

---

## Why This Is Not “Over-Engineering”

This might look like extra structure — but it actually **reduces complexity**:

* fewer conditionals later
* clearer node responsibilities
* safer automation
* easier onboarding for new contributors

This is how you build agents that **survive contact with real organizations**.


In [ ]:
"""Nodes for Experimentation Portfolio Orchestrator Agent

This module contains all the workflow nodes for the EPO agent.
Following the MVP-first approach: rule-based logic, no LLM dependencies.
"""

from typing import Dict, Any
from config import ExperimentationPortfolioOrchestratorState


def goal_node(state: ExperimentationPortfolioOrchestratorState) -> Dict[str, Any]:
    """
    Goal Node: Define the goal for experiment portfolio analysis.

    This is a simple rule-based goal definition that sets the framework
    for analyzing the experimentation portfolio.

    MVP: Fixed goal, no LLM needed.
    """
    experiment_id = state.get("experiment_id")

    # Build goal based on whether analyzing specific experiment or entire portfolio
    if experiment_id:
        goal = {
            "objective": f"Analyze experiment {experiment_id} and provide decision recommendation",
            "scope": "single_experiment",
            "experiment_id": experiment_id,
            "focus_areas": [
                "statistical_analysis",
                "causal_impact_estimation",
                "decision_recommendation",
                "roi_assessment"
            ]
        }
    else:
        goal = {
            "objective": "Analyze entire experimentation portfolio and provide executive summary",
            "scope": "portfolio_wide",
            "experiment_id": None,
            "focus_areas": [
                "portfolio_overview",
                "experiment_status_tracking",
                "roi_aggregation",
                "risk_assessment",
                "learning_extraction",
                "executive_reporting"
            ]
        }

    return {
        "goal": goal,
        "errors": state.get("errors", [])
    }


def planning_node(state: ExperimentationPortfolioOrchestratorState) -> Dict[str, Any]:
    """
    Planning Node: Create execution plan based on goal.

    This creates a step-by-step plan. Rule-based, no LLM needed.

    MVP: Template-based plan, no LLM needed.
    """
    goal = state.get("goal")

    if not goal:
        return {
            "errors": state.get("errors", []) + ["planning_node: goal is required"]
        }

    scope = goal.get("scope", "portfolio_wide")

    if scope == "single_experiment":
        # Plan for single experiment analysis
        plan = [
            {
                "step": 1,
                "name": "data_loading",
                "description": "Load experiment definitions, metrics, analysis, decisions, learnings, and audit log",
                "dependencies": [],
                "outputs": [
                    "experiment_definitions",
                    "experiment_metrics",
                    "experiment_analysis",
                    "experiment_decisions",
                    "experiment_learnings",
                    "experiment_audit_log",
                    "definitions_lookup",
                    "metrics_lookup",
                    "analysis_lookup",
                    "decisions_lookup",
                    "learnings_lookup",
                    "audit_log_lookup"
                ]
            },
            {
                "step": 2,
                "name": "statistical_analysis",
                "description": "Perform statistical analysis (if metrics exist but analysis missing)",
                "dependencies": ["data_loading"],
                "outputs": ["calculated_analyses"]
            },
            {
                "step": 3,
                "name": "decision_evaluation",
                "description": "Evaluate experiment and generate decision recommendation",
                "dependencies": ["statistical_analysis"],
                "outputs": ["generated_decisions"]
            },
            {
                "step": 4,
                "name": "roi_calculation",
                "description": "Calculate ROI and business value metrics",
                "dependencies": ["decision_evaluation"],
                "outputs": ["portfolio_roi", "performance_metrics"]
            },
            {
                "step": 5,
                "name": "report_generation",
                "description": "Generate executive-ready report",
                "dependencies": ["roi_calculation"],
                "outputs": ["portfolio_report", "report_file_path"]
            }
        ]
    else:
        # Plan for portfolio-wide analysis
        plan = [
            {
                "step": 1,
                "name": "data_loading",
                "description": "Load all experiment data: portfolio, definitions, metrics, analysis, decisions, learnings, audit log",
                "dependencies": [],
                "outputs": [
                    "portfolio",
                    "experiment_definitions",
                    "experiment_metrics",
                    "experiment_analysis",
                    "experiment_decisions",
                    "experiment_learnings",
                    "experiment_audit_log",
                    "portfolio_lookup",
                    "definitions_lookup",
                    "metrics_lookup",
                    "analysis_lookup",
                    "decisions_lookup",
                    "learnings_lookup",
                    "audit_log_lookup"
                ]
            },
            {
                "step": 2,
                "name": "portfolio_analysis",
                "description": "Analyze portfolio status and identify experiments needing analysis/decisions",
                "dependencies": ["data_loading"],
                "outputs": ["analyzed_experiments", "portfolio_summary"]
            },
            {
                "step": 3,
                "name": "statistical_analysis",
                "description": "Perform statistical analysis for experiments with metrics but missing analysis",
                "dependencies": ["portfolio_analysis"],
                "outputs": ["calculated_analyses"]
            },
            {
                "step": 4,
                "name": "decision_evaluation",
                "description": "Evaluate experiments and generate decision recommendations",
                "dependencies": ["statistical_analysis"],
                "outputs": ["generated_decisions"]
            },
            {
                "step": 5,
                "name": "portfolio_insights",
                "description": "Generate cross-experiment insights and patterns",
                "dependencies": ["decision_evaluation"],
                "outputs": ["portfolio_insights"]
            },
            {
                "step": 6,
                "name": "roi_calculation",
                "description": "Calculate portfolio-level ROI and business value metrics",
                "dependencies": ["portfolio_insights"],
                "outputs": ["portfolio_roi", "performance_metrics"]
            },
            {
                "step": 7,
                "name": "report_generation",
                "description": "Generate executive-ready portfolio report",
                "dependencies": ["roi_calculation"],
                "outputs": ["portfolio_report", "report_file_path"]
            }
        ]

    return {
        "plan": plan,
        "errors": state.get("errors", [])
    }
